## TYTAN tutorial 追加問題2（美術館）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）で美術館パズル（ライトアップ）を解く](https://vigne-cla.com/21-15/)

### 問題
今回は美術館（ライトアップ）と呼ばれるパズル。

ライトを置くと上下左右にビーム状に光が届く。ライトの光は他のライトに当たってはいけない。数字マスはその４近傍のライトの個数を指定している。部屋全体を明るくするにはどこにライトを置けば良いか？というもの。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-15_1r-200x200.png" width = 15%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-15_2r-200x200.png" width = 15%>
</div>

ブラウザ上で遊んでみると良い → [Puzzle Team](https://ja.puzzle-light-up.com/)

### このQUBO設定を使おう

<font color="red">「n個の量子ビットからm個を1にする」</font>

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

<font color="red">「ｎ個の量子ビットからｍ個またはｍ+1個を１にする」</font>

例）3個の量子ビットから1個または2個を1にする<sub>ちなみに東工大の化学の入試問題「次の記述のうち正しいものはどれか。正解は１つまたは２つある。」</sub>
```
H = (q1 + q2 + q3 - 1.5)**2
```

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

まず、白マスを量子ビットに対応させ、結果が１ならライトを置くことにする。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-15_3r.png" width = 13%>
</div>

赤線に着目すると、**各ラインには０個または１個のライトを置く必要がある。**１個置くのはまあ分かるが、実は０個の可能性もある。すべて脇から照らしてくれればOKなので。ライン上に２個以上置いてしまうと「ライトの光は他のライトに当たってはいけない」に反する。よって「０個または１個」の設定。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-15_4r.png" width = 38%>
</div>

次に、一つの白マスに着目して、**十字関係のマス（ライトが置かれる可能性があるマス）には１個または２個のライトがある必要がある。**少なくとも１個はないとこのマスが照らされず、よく考えると１個だけとは限らず、２個の可能性もある（光がクロスするのはOKなので）。３個以上はあり得ない。よって「１個または２個」の設定。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-15_5.png" width = 38%>
</div>

最後に、数字マスに着目して「４近傍からｍ個だけ1になる」を設定。ここはシンプルで分かりやすい。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-15_6.png" width = 38%>
</div>

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [3]:
from tytan import *

#量子ビットを用意する
q0 = symbols('q0')
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')
q5 = symbols('q5')
q6 = symbols('q6')

#横方向の各ライン、「0個または1個が1になる」＝「0.5個が1になる」
H = 0
H += (q2 + q3 + q4 - 0.5)**2
H += (q5 + q6 - 0.5)**2

#縦方向の各ライン、同様
H += (q0 + q2 - 0.5)**2
H += (q3 + q5 - 0.5)**2
H += (q1 + q4 + q6 - 0.5)**2

#各白マス、「1個または2個が1になる」＝「1.5個が1になる」
H += (q0 + q2 - 1.5)**2
H += (q1 + q4 + q6 - 1.5)**2
H += (q0 + q2 + q3 + q4 - 1.5)**2
H += (q2 + q3 + q4 + q5 - 1.5)**2
H += (q2 + q3 + q1 + q6 - 1.5)**2
H += (q3 + q5 + q6 - 1.5)**2
H += (q4 + q5 + q6 - 1.5)**2

#各数字マス
H += (q0 + q1 + q3 - 2)**2
H += (q2 + q5 - 1)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#結果
for r in result:
    print(r)

offset
22.0000000000000
[{'q0': 1, 'q1': 1, 'q2': 0, 'q3': 0, 'q4': 0, 'q5': 1, 'q6': 0}, -19.0, 81]
[{'q0': 1, 'q1': 0, 'q2': 0, 'q3': 1, 'q4': 0, 'q5': 0, 'q6': 1}, -18.0, 19]


最適解を白マスに戻してやると模範解答の通りである。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-15_2r-200x200.png" width = 15%>
</div>